In [1]:
import gc
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

from scipy.io import loadmat
from shapely.geometry import Point
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from torchinfo import summary

%matplotlib inline

In [2]:
foo_dspl_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_dspl'
foo_dsplRadial_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_dsplRadial'
foo_trac_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_trac'
foo_tracRadial_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_tracRadial'

In [3]:
def data_to_npArrays(dspl_path, dsplRadial_path, trac_path, tracRadial_path):
    number_samples = len([name for name in os.listdir(dspl_path) if os.path.isfile(os.path.join(dspl_path, name))])
    number_radials = len([name for name in os.listdir(dsplRadial_path) if os.path.isfile(os.path.join(dsplRadial_path, name))])
    
    # save all samples in matrix
    samples = [] 
    for i, filename in enumerate(os.listdir(dspl_path)):
        f = os.path.join(dspl_path, filename)
        if os.path.isfile(f):
            sample = loadmat(f)
            if '__header__' in sample: del sample['__header__']
            if '__version__' in sample: del sample['__version__']
            if '__globals__' in sample: del sample['__globals__']
            sample['name'] = filename
            samples = np.append(samples, sample)
        else:
            continue
    samples = np.array(samples)

    # save all radial patterns of displacements in matrix
    dspl_radials = []
    for i, filename in enumerate(os.listdir(dsplRadial_path)):
        f = os.path.join(dsplRadial_path, filename)
        if os.path.isfile(f):
            radial = loadmat(f)
            if '__header__' in radial: del radial['__header__']
            if '__version__' in radial: del radial['__version__']
            if '__globals__' in radial: del radial['__globals__']
            radial['name'] = filename
            dspl_radials = np.append(dspl_radials, radial)
        else:
            continue
    dspl_radials = np.array(dspl_radials)
    
    # save all targets in matrix
    targets = []
    for i, filename in enumerate(os.listdir(trac_path)):
        f = os.path.join(trac_path, filename)
        if os.path.isfile(f):
            target = loadmat(f)
            if '__header__' in target: del target['__header__']
            if '__version__' in target: del target['__version__']
            if '__globals__' in target: del target['__globals__']
            target['name'] = filename
            targets = np.append(targets, target)
        else:
            continue 
    targets = np.array(targets)
    
    # save all radial patterns of traction forces in matrix
    trac_radials = []
    for i, filename in enumerate(os.listdir(tracRadial_path)):
        f = os.path.join(tracRadial_path, filename)
        if os.path.isfile(f):
            radial = loadmat(f)
            if '__header__' in radial: del radial['__header__']
            if '__version__' in radial: del radial['__version__']
            if '__globals__' in radial: del radial['__globals__']
            radial['name'] = filename
            trac_radials = np.append(trac_radials, radial)
        else:
            continue
    trac_radials = np.array(trac_radials)

    return samples, dspl_radials, targets, trac_radials

In [4]:
samples, dspl_radials, targets, trac_radials = data_to_npArrays(foo_dspl_path, foo_dsplRadial_path, foo_trac_path, foo_tracRadial_path)
samples, targets = np.append(samples, dspl_radials), np.append(targets, trac_radials)

Compute loss between sample and target

In [5]:
# Currently, X and y are of shape (samples, width, height, depth)
X, y = np.array([sample['dspl'] for sample in samples]), np.array([target['trac'] for target in targets])
# Reshape to (samples, channels, depth, height, width)
X = np.moveaxis(X[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
y = np.moveaxis(y[:, np.newaxis], [2, 3, 4], [-1, 3, 2])

In [6]:
X.shape

(1, 1, 2, 104, 104)

In [7]:
x_min = X.min(axis=(3, 4), keepdims=True)
x_max = X.max(axis=(3, 4), keepdims=True)

X = (X - x_min)/(x_max-x_min)

y_min = y.min(axis=(3, 4), keepdims=True)
y_max = y.max(axis=(3, 4), keepdims=True)

y = (y - y_min)/(y_max-y_min)

In [43]:
def forward(predictions, target):
    loss_fn = nn.MSELoss(reduction = 'sum')
    loss = 0.5 * loss_fn(predictions, target)
    return loss

In [56]:
0.5 * np.sum(np.power(X-y,2))

138.6464594959187

In [9]:
X_train = torch.from_numpy(X).double()
y_train = torch.from_numpy(y).double()

In [11]:
class ConvBlock_1(nn.Module):
    """Conv3D -> BatchNorm -> ReLU"""
    
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=(2,3,3), padding='same'),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv_block_1(x)

In [12]:
class ConvBlock_2(nn.Module):
    """Conv3D -> ReLU"""
    
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block_2 = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=(2,3,3), padding='same'),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv_block_2(x)

In [13]:
class TracNet(nn.Module):
    def __init__(self, n_channels):
        super().__init__()
        
        self.s1 = ConvBlock_1(n_channels, 32)
        self.s2 = nn.Conv3d(32, 1, kernel_size=(2, 3, 3), padding='same')
        
    def forward(self, x):
        x1 = self.s1(x)
        logits = self.s2(x1)
        return logits

In [14]:
def initialize_weight(module):
    if isinstance(module, (nn.Conv3d, nn.ConvTranspose3d)):
        nn.init.xavier_normal_(module.weight)

In [21]:
class Custom_MSE(nn.Module):
    def __init__(self):
        super(Custom_MSE, self).__init__();
    
    def forward(self, predictions, target):
        loss_fn = nn.MSELoss(reduction = 'sum')
        loss = 0.5 * loss_fn(predictions, target)
        return loss

In [39]:
def fit(model, optimizer, dataloaders, num_epochs):
    model = model.double()
    loss_fn = Custom_MSE()
    for epoch in range(num_epochs):
        # Training
        model.train()
        for xb, yb in dataloaders["train"]:
            logits = model(xb)
            logits = logits.double()     
            loss = loss_fn(logits, yb)
            model.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Evaluation
        model.eval()
        loss = 0
        with torch.no_grad():
            for xb, yb in dataloaders["val"]:
                xb = xb.to(device)
                yb = yb.to(device)
                logits = model(xb)
                logits = logits.double()
                loss += loss_fn.forward(logits, yb)
            avg_val_loss = loss / len(dataloaders["val"])
        print(f"Epoch {epoch}: test_loss = {avg_val_loss}")

In [37]:
model = TracNet(n_channels=1)
print(summary(model, verbose=2))

model.apply(initialize_weight)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0000001, momentum=0.9)

fit(model, optimizer, dataloaders, num_epochs=2000)

Layer (type:depth-idx)                   Param #
TracNet                                  --
├─ConvBlock_1: 1-1                       --
│    └─conv_block_1.0.weight             ├─576
│    └─conv_block_1.0.bias               ├─32
│    └─conv_block_1.1.weight             ├─32
│    └─conv_block_1.1.bias               └─32
│    └─Sequential: 2-1                   --
│    │    └─0.weight                     ├─576
│    │    └─0.bias                       ├─32
│    │    └─1.weight                     ├─32
│    │    └─1.bias                       └─32
│    │    └─Conv3d: 3-1                  608
│    │    │    └─weight                  ├─576
│    │    │    └─bias                    └─32
│    │    └─BatchNorm3d: 3-2             64
│    │    │    └─weight                  ├─32
│    │    │    └─bias                    └─32
│    │    └─ReLU: 3-3                    --
├─Conv3d: 1-2                            577
│    └─weight                            ├─576
│    └─bias                            

Epoch 182: train_loss = 21.179
Epoch 183: train_loss = 21.117
Epoch 184: train_loss = 21.055
Epoch 185: train_loss = 20.994
Epoch 186: train_loss = 20.933
Epoch 187: train_loss = 20.873
Epoch 188: train_loss = 20.814
Epoch 189: train_loss = 20.755
Epoch 190: train_loss = 20.697
Epoch 191: train_loss = 20.639
Epoch 192: train_loss = 20.582
Epoch 193: train_loss = 20.525
Epoch 194: train_loss = 20.469
Epoch 195: train_loss = 20.413
Epoch 196: train_loss = 20.358
Epoch 197: train_loss = 20.303
Epoch 198: train_loss = 20.249
Epoch 199: train_loss = 20.196
Epoch 200: train_loss = 20.143
Epoch 201: train_loss = 20.090
Epoch 202: train_loss = 20.038
Epoch 203: train_loss = 19.986
Epoch 204: train_loss = 19.935
Epoch 205: train_loss = 19.884
Epoch 206: train_loss = 19.834
Epoch 207: train_loss = 19.784
Epoch 208: train_loss = 19.734
Epoch 209: train_loss = 19.685
Epoch 210: train_loss = 19.636
Epoch 211: train_loss = 19.588
Epoch 212: train_loss = 19.540
Epoch 213: train_loss = 19.493
Epoch 21

Epoch 449: train_loss = 13.547
Epoch 450: train_loss = 13.533
Epoch 451: train_loss = 13.518
Epoch 452: train_loss = 13.504
Epoch 453: train_loss = 13.489
Epoch 454: train_loss = 13.475
Epoch 455: train_loss = 13.461
Epoch 456: train_loss = 13.446
Epoch 457: train_loss = 13.432
Epoch 458: train_loss = 13.418
Epoch 459: train_loss = 13.404
Epoch 460: train_loss = 13.390
Epoch 461: train_loss = 13.376
Epoch 462: train_loss = 13.361
Epoch 463: train_loss = 13.347
Epoch 464: train_loss = 13.333
Epoch 465: train_loss = 13.319
Epoch 466: train_loss = 13.305
Epoch 467: train_loss = 13.291
Epoch 468: train_loss = 13.278
Epoch 469: train_loss = 13.264
Epoch 470: train_loss = 13.250
Epoch 471: train_loss = 13.236
Epoch 472: train_loss = 13.222
Epoch 473: train_loss = 13.209
Epoch 474: train_loss = 13.195
Epoch 475: train_loss = 13.181
Epoch 476: train_loss = 13.168
Epoch 477: train_loss = 13.154
Epoch 478: train_loss = 13.141
Epoch 479: train_loss = 13.127
Epoch 480: train_loss = 13.114
Epoch 48

Epoch 716: train_loss = 10.652
Epoch 717: train_loss = 10.644
Epoch 718: train_loss = 10.635
Epoch 719: train_loss = 10.627
Epoch 720: train_loss = 10.619
Epoch 721: train_loss = 10.611
Epoch 722: train_loss = 10.603
Epoch 723: train_loss = 10.595
Epoch 724: train_loss = 10.587
Epoch 725: train_loss = 10.579
Epoch 726: train_loss = 10.571
Epoch 727: train_loss = 10.563
Epoch 728: train_loss = 10.555
Epoch 729: train_loss = 10.547
Epoch 730: train_loss = 10.539
Epoch 731: train_loss = 10.531
Epoch 732: train_loss = 10.523
Epoch 733: train_loss = 10.515
Epoch 734: train_loss = 10.507
Epoch 735: train_loss = 10.499
Epoch 736: train_loss = 10.492
Epoch 737: train_loss = 10.484
Epoch 738: train_loss = 10.476
Epoch 739: train_loss = 10.468
Epoch 740: train_loss = 10.460
Epoch 741: train_loss = 10.453
Epoch 742: train_loss = 10.445
Epoch 743: train_loss = 10.437
Epoch 744: train_loss = 10.430
Epoch 745: train_loss = 10.422
Epoch 746: train_loss = 10.414
Epoch 747: train_loss = 10.407
Epoch 74

Epoch 989: train_loss = 8.870
Epoch 990: train_loss = 8.864
Epoch 991: train_loss = 8.859
Epoch 992: train_loss = 8.854
Epoch 993: train_loss = 8.849
Epoch 994: train_loss = 8.843
Epoch 995: train_loss = 8.838
Epoch 996: train_loss = 8.833
Epoch 997: train_loss = 8.828
Epoch 998: train_loss = 8.823
Epoch 999: train_loss = 8.817
Epoch 1000: train_loss = 8.812
Epoch 1001: train_loss = 8.807
Epoch 1002: train_loss = 8.802
Epoch 1003: train_loss = 8.797
Epoch 1004: train_loss = 8.791
Epoch 1005: train_loss = 8.786
Epoch 1006: train_loss = 8.781
Epoch 1007: train_loss = 8.776
Epoch 1008: train_loss = 8.771
Epoch 1009: train_loss = 8.766
Epoch 1010: train_loss = 8.760
Epoch 1011: train_loss = 8.755
Epoch 1012: train_loss = 8.750
Epoch 1013: train_loss = 8.745
Epoch 1014: train_loss = 8.740
Epoch 1015: train_loss = 8.735
Epoch 1016: train_loss = 8.730
Epoch 1017: train_loss = 8.725
Epoch 1018: train_loss = 8.720
Epoch 1019: train_loss = 8.715
Epoch 1020: train_loss = 8.710
Epoch 1021: train_l

Epoch 1256: train_loss = 7.682
Epoch 1257: train_loss = 7.678
Epoch 1258: train_loss = 7.674
Epoch 1259: train_loss = 7.671
Epoch 1260: train_loss = 7.667
Epoch 1261: train_loss = 7.663
Epoch 1262: train_loss = 7.660
Epoch 1263: train_loss = 7.656
Epoch 1264: train_loss = 7.652
Epoch 1265: train_loss = 7.649
Epoch 1266: train_loss = 7.645
Epoch 1267: train_loss = 7.641
Epoch 1268: train_loss = 7.638
Epoch 1269: train_loss = 7.634
Epoch 1270: train_loss = 7.630
Epoch 1271: train_loss = 7.627
Epoch 1272: train_loss = 7.623
Epoch 1273: train_loss = 7.619
Epoch 1274: train_loss = 7.616
Epoch 1275: train_loss = 7.612
Epoch 1276: train_loss = 7.608
Epoch 1277: train_loss = 7.605
Epoch 1278: train_loss = 7.601
Epoch 1279: train_loss = 7.597
Epoch 1280: train_loss = 7.594
Epoch 1281: train_loss = 7.590
Epoch 1282: train_loss = 7.587
Epoch 1283: train_loss = 7.583
Epoch 1284: train_loss = 7.579
Epoch 1285: train_loss = 7.576
Epoch 1286: train_loss = 7.572
Epoch 1287: train_loss = 7.569
Epoch 12

Epoch 1523: train_loss = 6.821
Epoch 1524: train_loss = 6.818
Epoch 1525: train_loss = 6.815
Epoch 1526: train_loss = 6.812
Epoch 1527: train_loss = 6.810
Epoch 1528: train_loss = 6.807
Epoch 1529: train_loss = 6.804
Epoch 1530: train_loss = 6.801
Epoch 1531: train_loss = 6.799
Epoch 1532: train_loss = 6.796
Epoch 1533: train_loss = 6.793
Epoch 1534: train_loss = 6.790
Epoch 1535: train_loss = 6.788
Epoch 1536: train_loss = 6.785
Epoch 1537: train_loss = 6.782
Epoch 1538: train_loss = 6.779
Epoch 1539: train_loss = 6.777
Epoch 1540: train_loss = 6.774
Epoch 1541: train_loss = 6.771
Epoch 1542: train_loss = 6.768
Epoch 1543: train_loss = 6.766
Epoch 1544: train_loss = 6.763
Epoch 1545: train_loss = 6.760
Epoch 1546: train_loss = 6.758
Epoch 1547: train_loss = 6.755
Epoch 1548: train_loss = 6.752
Epoch 1549: train_loss = 6.749
Epoch 1550: train_loss = 6.747
Epoch 1551: train_loss = 6.744
Epoch 1552: train_loss = 6.741
Epoch 1553: train_loss = 6.739
Epoch 1554: train_loss = 6.736
Epoch 15

Epoch 1790: train_loss = 6.174
Epoch 1791: train_loss = 6.172
Epoch 1792: train_loss = 6.170
Epoch 1793: train_loss = 6.168
Epoch 1794: train_loss = 6.166
Epoch 1795: train_loss = 6.164
Epoch 1796: train_loss = 6.162
Epoch 1797: train_loss = 6.160
Epoch 1798: train_loss = 6.157
Epoch 1799: train_loss = 6.155
Epoch 1800: train_loss = 6.153
Epoch 1801: train_loss = 6.151
Epoch 1802: train_loss = 6.149
Epoch 1803: train_loss = 6.147
Epoch 1804: train_loss = 6.145
Epoch 1805: train_loss = 6.143
Epoch 1806: train_loss = 6.141
Epoch 1807: train_loss = 6.139
Epoch 1808: train_loss = 6.137
Epoch 1809: train_loss = 6.134
Epoch 1810: train_loss = 6.132
Epoch 1811: train_loss = 6.130
Epoch 1812: train_loss = 6.128
Epoch 1813: train_loss = 6.126
Epoch 1814: train_loss = 6.124
Epoch 1815: train_loss = 6.122
Epoch 1816: train_loss = 6.120
Epoch 1817: train_loss = 6.118
Epoch 1818: train_loss = 6.116
Epoch 1819: train_loss = 6.114
Epoch 1820: train_loss = 6.112
Epoch 1821: train_loss = 6.110
Epoch 18